In [1]:
import pickle
import bz2
import gzip
import lzma
import pickle

In [2]:
with open('emorec_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [3]:
data.head()

,subid,subage,subgender,condition,heartbeats (actual) mean,IAcc,IAcc HighvsLow,sadnessAcc,neutralAcc,happinessAcc,angerAcc,emorec total
0,1,28,F,control,67.6,0.97,High IAcc,0.85,0.57,0.50,0.50,2.42
1,2,28,F,self,75.6,0.75,High IAcc,0.50,0.60,0.45,0.50,2.05
2,3,25,F,self,75.0,0.70,High IAcc,1.00,0.87,0.37,0.00,2.24
3,4,26,F,self,84.3,0.91,High IAcc,0.55,0.22,0.42,0.14,1.33
4,5,30,F,self,67.3,0.81,High IAcc,0.60,0.42,0.37,0.42,1.81


In [4]:
# open participants_compressed.pbz2
with bz2.open('participants_compressed.pbz2', 'rb') as f:
    participants = pickle.load(f)

In [5]:
participants.head()
data = participants
data.head()

,TIME,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID,FPOGV,LPUPILD,LPUPILV,RPUPILD,RPUPILV,USER,FileName
2883,148.15216,0.49825,0.46377,147.79085,0.36131,257.0,1.0,0.00574,1.0,0.00581,1.0,STIMULI_ONSET1,emorecm4623_e
2884,148.17009,0.49826,0.46133,147.79085,0.37924,257.0,1.0,0.00568,1.0,0.00584,1.0,STIMULI_ONSET1,emorecm4623_e
2885,148.18607,0.49806,0.45905,147.79085,0.39522,257.0,1.0,0.00571,1.0,0.00584,1.0,STIMULI_ONSET1,emorecm4623_e
2886,148.20239,0.49787,0.45710,147.79085,0.41154,257.0,1.0,0.00571,1.0,0.00583,1.0,STIMULI_ONSET1,emorecm4623_e
2887,148.22081,0.49771,0.45584,147.79085,0.42996,257.0,1.0,0.00568,1.0,0.00581,1.0,STIMULI_ONSET1,emorecm4623_e


In [7]:
data.shape

(3409501, 13)

In [9]:
data.query("USER=='STIMULI_ONSET1'")

,TIME,FPOGX,FPOGY,FPOGS,FPOGD,FPOGID,FPOGV,LPUPILD,LPUPILV,RPUPILD,RPUPILV,USER,FileName
2883,148.15216,0.49825,0.46377,147.79085,0.36131,257.0,1.0,0.00574,1.0,0.00581,1.0,STIMULI_ONSET1,emorecm4623_e
2884,148.17009,0.49826,0.46133,147.79085,0.37924,257.0,1.0,0.00568,1.0,0.00584,1.0,STIMULI_ONSET1,emorecm4623_e
2885,148.18607,0.49806,0.45905,147.79085,0.39522,257.0,1.0,0.00571,1.0,0.00584,1.0,STIMULI_ONSET1,emorecm4623_e
2886,148.20239,0.49787,0.45710,147.79085,0.41154,257.0,1.0,0.00571,1.0,0.00583,1.0,STIMULI_ONSET1,emorecm4623_e
2887,148.22081,0.49771,0.45584,147.79085,0.42996,257.0,1.0,0.00568,1.0,0.00581,1.0,STIMULI_ONSET1,emorecm4623_e
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4143694,183.09001,0.53867,0.34558,182.48216,0.60785,316.0,1.0,0.00749,1.0,0.00776,1.0,STIMULI_ONSET1,emorec_ctrlF6721_e
4143695,183.10629,0.53871,0.35091,182.48216,0.62413,316.0,1.0,0.00737,1.0,0.00776,1.0,STIMULI_ONSET1,emorec_ctrlF6721_e
4143696,183.12395,0.54272,0.38556,182.48216,0.62413,316.0,0.0,0.00743,1.0,0.00761,1.0,STIMULI_ONSET1,emorec_ctrlF6721_e
4143697,183.13918,0.54272,0.38556,182.48216,0.62413,316.0,0.0,0.00743,1.0,0.00761,1.0,STIMULI_ONSET1,emorec_ctrlF6721_e


In [6]:
import pandas as pd
import math

# Specify fixation radius and minimum duration
fixation_radius = 58
min_duration = 0.12

# Get unique trial names
trials = data['USER'].unique()

# Loop through trials and calculate fixation count and duration
for trial in trials:
    trial_data = data[data['USER'] == trial]
    fixation_count = 0
    fixation_duration = 0
    last_fixation_end_time = float('nan')
    last_x = last_y = 0

    for index, row in trial_data.iterrows():
        x = row['FPOGX'] * 1920  # Convert fraction of screen to pixels
        y = row['FPOGY'] * 1080  # Convert fraction of screen to pixels

        if math.isnan(last_fixation_end_time):
            # First sample in trial
            fixation_start_time = row['FPOGS']
            last_fixation_end_time = fixation_start_time
        elif math.sqrt((x - last_x) ** 2 + (y - last_y) ** 2) <= fixation_radius and \
                row['FPOGS'] - last_fixation_end_time <= min_duration:
            # Continuing fixation
            pass
        elif math.sqrt((x - last_x) ** 2 + (y - last_y) ** 2) <= fixation_radius and \
                row['FPOGS'] - last_fixation_end_time > min_duration:
            # Extended fixation
            fixation_duration += row['FPOGS'] - last_fixation_end_time
            last_fixation_end_time = row['FPOGS'] + row['FPOGD']
        elif math.sqrt((x - last_x) ** 2 + (y - last_y) ** 2) > fixation_radius:
            # New fixation
            fixation_count += 1
            fixation_duration += row['FPOGD']
            fixation_start_time = row['FPOGS']
            last_fixation_end_time = row['FPOGS'] + row['FPOGD']

        last_x = x
        last_y = y

    # Print results for trial
    print(f"Trial: {trial}, Fixation Count: {fixation_count}, Fixation Duration: {fixation_duration:.2f} seconds")

KeyboardInterrupt: 